---
title: "Binary representation table"
format: html
execute:
    eval: true
jupyter: IRkernel
---

In [34]:
library(jsonlite)
library(DT)
library(IRdisplay)
library(htmlwidgets)

In [35]:
config_dir <- "../phenoRankeR/inst/extdata/config"
json_data <- fromJSON(file.path(config_dir, "pheno_blast_col_colors.json"))

In [59]:
user <- "pheno.ranker@playground.com"
run_id <- "20241006015118" #working
input_format <- "bff" #working

run_id <- "20241006003308" #not working
input_format <- "json" #not working

user_dir <- file.path("../data/user_data", user)
out_dir <- file.path(user_dir, "output/rankedPatients", run_id)
file_suffix <- "_alignment.csv"

file_path <- file.path(out_dir, paste0(run_id, file_suffix))
print(file_path)

# binary representation of each patient
bin_df <- read.table(
    file_path,
    header = TRUE,
    sep = ";",
    row.names = NULL
)
head(bin_df)

[1] "../data/user_data/pheno.ranker@playground.com/output/rankedPatients/20241006003308/20241006003308_alignment.csv"


,Id,X18.24,X30.34,X45.49,X50.54,X55.59,X60.64,X65.69,X70.74,X75.79,⋯,id.PR_00000016,id.PR_00000017,id.PR_00000018,id.PR_00000019,id.PR_00000020,id.PR_00000021,id.PR_00000022,id.PR_00000023,id.PR_00000024,id.PR_00000025
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,Id,Age_Category.18-24,Age_Category.30-34,Age_Category.45-49,Age_Category.50-54,Age_Category.55-59,Age_Category.60-64,Age_Category.65-69,Age_Category.70-74,Age_Category.75-79,⋯,id.PR_00000016,id.PR_00000017,id.PR_00000018,id.PR_00000019,id.PR_00000020,id.PR_00000021,id.PR_00000022,id.PR_00000023,id.PR_00000024,id.PR_00000025
2,T|PR_00000001,0,0,0,0,0,0,0,1,0,⋯,0,0,0,0,0,0,0,0,0,0
3,R|PR_00000001,0,0,0,0,0,0,0,1,0,⋯,0,0,0,0,0,0,0,0,0,0
4,R|PR_00000010,0,0,0,0,0,0,0,1,0,⋯,0,0,0,0,0,0,0,0,0,0
5,R|PR_00000015,0,0,0,0,0,0,0,1,0,⋯,0,0,0,0,0,0,0,0,0,0
6,R|PR_00000016,0,0,0,0,0,0,0,1,0,⋯,1,0,0,0,0,0,0,0,0,0


In [60]:
# Function to generate HSLA colors with variable hue
generate_hsla_colors <- function(h_start, h_end, fixed_s = 30, fixed_l = 80, fixed_a = 1) {
  # Create a vector of hues in the range [h_start, h_end]
  hues <- seq(h_start, h_end, by = 1)
  
  # Generate HSLA color strings
  hsla_colors <- paste0("hsla(", hues, ", ", fixed_s, "%, ", fixed_l, "%, ", fixed_a, ")")
  
  return(hsla_colors)
}

hsla_colors <- generate_hsla_colors(1, 360)

In [106]:
json_path_row <- bin_df[1,]

#remove everything after the first dot
top_level_row <- gsub("\\..*", "", json_path_row)
top_level_row[1] <- "top_level"

# add the top level row to the data frame
bin_df2 <- rbind(top_level_row, bin_df)

# Separate the first column and the rest of the data frame
first_column <- bin_df2[, 1]
rest_of_df <- bin_df2[, -1]

# Simplify the first row of the remaining columns to ensure no lists
simplified_first_row <- sapply(rest_of_df[1, ], as.character)

# Sort the remaining columns based on the simplified first row
sorted_rest_of_df <- rest_of_df[, order(simplified_first_row)]

# Recombine the first column with the sorted remaining columns
bin_df2 <- cbind(first_column, sorted_rest_of_df)

# Rename the first column back to "Id"
names(bin_df2)[1] <- "Id"

top_level_row <- unlist(bin_df2[1,])

color_map <- NULL
# color_map <- json_data[[input_format]]
if (is.null(color_map)) {
  print("No color map found, generating random colors")
  top_levels <- unique(top_level_row[-1])
  # print("Top levels:")
  # print(top_levels)
  colors <- sample(hsla_colors, length(top_levels))
  # print("Colors:")
  # print(colors)
  # create the color mapping in form of a list
  color_map <- list()
  for (i in 1:length(top_levels)) {
    color_map[top_levels[i]] <- colors[i]
  }
}

print("Color map:")
print(color_map)

# add a new row containing the colors
color_row <- unlist(sapply(
  bin_df2[1,],
  function(x) color_map[[x]]
))
color_row <- c("background-color", color_row)

# add the color row to the data frame
bin_df2 <- rbind(color_row, bin_df2)

head(bin_df2)

[1] "simplified_first_row"
                 X18.24                  X30.34                  X45.49 
         "Age_Category"          "Age_Category"          "Age_Category" 
                 X50.54                  X55.59                  X60.64 
         "Age_Category"          "Age_Category"          "Age_Category" 
                 X65.69                  X70.74                  X75.79 
         "Age_Category"          "Age_Category"          "Age_Category" 
                   X80.                      No                     Yes 
         "Age_Category"             "Arthritis"             "Arthritis" 
   X5.or.more.years.ago Within.the.past.2.years    Within.the.past.year 
              "Checkup"               "Checkup"               "Checkup" 
                   No.1                   Yes.1                    No.2 
           "Depression"            "Depression"              "Diabetes" 
                  Yes.2                    No.3                   Yes.3 
             "Diabetes" 

,Id,X18.24,X30.34,X45.49,X50.54,X55.59,X60.64,X65.69,X70.74,X75.79,⋯,id.PR_00000022,id.PR_00000023,id.PR_00000024,id.PR_00000025,No.5,Yes.5,Female,Male,No.6,Yes.6
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,background-color,"hsla(46, 30%, 80%, 1)","hsla(46, 30%, 80%, 1)","hsla(46, 30%, 80%, 1)","hsla(46, 30%, 80%, 1)","hsla(46, 30%, 80%, 1)","hsla(46, 30%, 80%, 1)","hsla(46, 30%, 80%, 1)","hsla(46, 30%, 80%, 1)","hsla(46, 30%, 80%, 1)",⋯,"hsla(238, 30%, 80%, 1)","hsla(238, 30%, 80%, 1)","hsla(238, 30%, 80%, 1)","hsla(238, 30%, 80%, 1)","hsla(350, 30%, 80%, 1)","hsla(350, 30%, 80%, 1)","hsla(129, 30%, 80%, 1)","hsla(129, 30%, 80%, 1)","hsla(28, 30%, 80%, 1)","hsla(28, 30%, 80%, 1)"
2,top_level,Age_Category,Age_Category,Age_Category,Age_Category,Age_Category,Age_Category,Age_Category,Age_Category,Age_Category,⋯,id,id,id,id,Other_Cancer,Other_Cancer,Sex,Sex,Skin_Cancer,Skin_Cancer
3,Id,Age_Category.18-24,Age_Category.30-34,Age_Category.45-49,Age_Category.50-54,Age_Category.55-59,Age_Category.60-64,Age_Category.65-69,Age_Category.70-74,Age_Category.75-79,⋯,id.PR_00000022,id.PR_00000023,id.PR_00000024,id.PR_00000025,Other_Cancer.No,Other_Cancer.Yes,Sex.Female,Sex.Male,Skin_Cancer.No,Skin_Cancer.Yes
4,T|PR_00000001,0,0,0,0,0,0,0,1,0,⋯,0,0,0,0,1,0,1,0,1,0
5,R|PR_00000001,0,0,0,0,0,0,0,1,0,⋯,0,0,0,0,1,0,1,0,1,0
6,R|PR_00000010,0,0,0,0,0,0,0,1,0,⋯,0,0,0,0,1,0,1,0,1,0


In [99]:
# extra header row containing the top level value
# while its cells are merged by top level

top_level_to_count <- table(top_level_row[-1])
top_level_to_count

top_level_to_color <- setNames(
  unique(color_row[-1]),
  unique(top_level_row[-1])
)

top_level_to_color

header_row <- paste(
  sapply(names(top_level_to_count), function(name) {
    colspan_value <- top_level_to_count[[name]]
    color <- top_level_to_color[[name]]

    if(name == "Other_Cancer") {
      print("Other_Cancer")
      print(colspan_value)
      print(color)
    }

    sprintf(
      '<th colspan="%d" style="background-color:%s;">%s</th>',
      colspan_value, color, name
    )
  }),
  collapse = ""  # Collapse into a single string
)
header_row


  Age_Category      Arthritis        Checkup     Depression       Diabetes 
            10              2              3              2              2 
      Exercise General_Health  Heart_Disease             id   Other_Cancer 
             2              5              2             25              2 
           Sex    Skin_Cancer 
             2              2 

Age_Category                Arthritis                  Checkup 
 "hsla(46, 30%, 80%, 1)" "hsla(235, 30%, 80%, 1)"  "hsla(91, 30%, 80%, 1)" 
              Depression                 Diabetes                 Exercise 
 "hsla(66, 30%, 80%, 1)"  "hsla(85, 30%, 80%, 1)" "hsla(317, 30%, 80%, 1)" 
          General_Health            Heart_Disease             Other_Cancer 
"hsla(220, 30%, 80%, 1)"  "hsla(50, 30%, 80%, 1)" "hsla(105, 30%, 80%, 1)" 
                     Sex              Skin_Cancer                       id 
 "hsla(73, 30%, 80%, 1)" "hsla(190, 30%, 80%, 1)" "hsla(100, 30%, 80%, 1)"

[1] "Other_Cancer"
[1] 2
[1] "hsla(105, 30%, 80%, 1)"


[1] "<th colspan=\"10\" style=\"background-color:hsla(46, 30%, 80%, 1);\">Age_Category</th><th colspan=\"2\" style=\"background-color:hsla(235, 30%, 80%, 1);\">Arthritis</th><th colspan=\"3\" style=\"background-color:hsla(91, 30%, 80%, 1);\">Checkup</th><th colspan=\"2\" style=\"background-color:hsla(66, 30%, 80%, 1);\">Depression</th><th colspan=\"2\" style=\"background-color:hsla(85, 30%, 80%, 1);\">Diabetes</th><th colspan=\"2\" style=\"background-color:hsla(317, 30%, 80%, 1);\">Exercise</th><th colspan=\"5\" style=\"background-color:hsla(220, 30%, 80%, 1);\">General_Health</th><th colspan=\"2\" style=\"background-color:hsla(50, 30%, 80%, 1);\">Heart_Disease</th><th colspan=\"25\" style=\"background-color:hsla(100, 30%, 80%, 1);\">id</th><th colspan=\"2\" style=\"background-color:hsla(105, 30%, 80%, 1);\">Other_Cancer</th><th colspan=\"2\" style=\"background-color:hsla(73, 30%, 80%, 1);\">Sex</th><th colspan=\"2\" style=\"background-color:hsla(190, 30%, 80%, 1);\">Skin_Cancer</th>"

In [98]:
# JavaScript to prepend the header row
headerCallback <- JS(
  "function(thead, data, start, end, display) {",
  sprintf("$(thead).closest('thead').prepend('<tr><th></th>%s</tr>');", header_row),
  "}"
)


# JavaScript to color the columns based on the values in the 2nd row of the data
initComplete <- JS(
  "function(settings, json) {",
  "$('tbody tr').each(function() {",
    # Loop over each column to set background color based on the value in the first row (background-color)
    "for (var i = 0; i < $('tbody tr:eq(0) td').length; i++) {",
      "var color = $('tbody tr:eq(0)').find('td:eq(' + i + ')').text();",
      "console.log(color);",

      # Apply background color to the cell
      "$('tbody tr').find('td:eq(' + i + ')').css('background-color', color);",
      "$('thead th.sorting_disabled').eq(i).css('background-color', color);",  # Apply background color to the header

      # hide the row with the colors
      "$('tbody tr').eq(0).hide();",
    "}",
  "});",
  "}"
)

# remove row 2 and 3
bin_df3 <- bin_df2[-c(2, 3),]

dt <- datatable(
  bin_df3,
  rownames = FALSE,
  escape = FALSE,
  options = list(
    scrollY = "500px",
    scrollX = TRUE,
    fixedHeader = TRUE,
    fixedColumns = list(leftColumns = 1),
    paging = FALSE,
    searching = FALSE,
    ordering = FALSE,
    headerCallback = headerCallback,
    initComplete = initComplete
  )
)

saved_wiget_path <- "notebooks/saved_widgets"
widget_fn <- "binary_representation.html"

saveWidget(
  dt,
  file.path(
    saved_wiget_path,
    widget_fn
  )
)


In [72]:
# make sure that vscode is running the live server
live_server_path <- "http://localhost:5500/jupyrenv/notebooks/saved_widgets"
iframe_src <- paste0(
  live_server_path,
  "/",
  widget_fn
)

display_html(
  paste0(
    "<iframe src='",
    iframe_src,
    "' width='100%' height='500px'></iframe>"
  )
)